In [ ]:
import os
import pandas as pd
import numpy as np
from random import randint
from nltk.corpus import wordnet

import nltk
nltk.download('wordnet')

!pip install textstat
import textstat
import nltk.data
nltk.download('punkt')
import random
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!pip install --upgrade pip  # ensures that pip is current
!git clone https://github.com/google-research/bleurt.git

fatal: destination path 'bleurt' already exists and is not an empty directory.


In [ ]:
!wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20-D6.zip .
!unzip BLEURT-20-D6.zip

--2023-08-05 21:40:25--  https://storage.googleapis.com/bleurt-oss-21/BLEURT-20-D6.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 171135344 (163M) [application/octet-stream]
Saving to: ‘BLEURT-20-D6.zip.1’

BLEURT-20-D6.zip.1  100%[===================>] 163.21M  42.2MB/s    in 4.6s    

2023-08-05 21:40:30 (35.8 MB/s) - ‘BLEURT-20-D6.zip.1’ saved [171135344/171135344]

--2023-08-05 21:40:30--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2023-08-05 21:40:30--
Total wall clock time: 4.8s
Downloaded: 1 files, 163M in 4.6s (35.8 MB/s)
Archive:  BLEURT-20-D6.zip
replace BLEURT-20-D6/bert_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
df_train = pd.read_csv('train_df.csv')
df_dev = pd.read_csv('dev_set.csv')


## Augment the Translations with BERT bases synonyms

In [ ]:
!pip install datasets
!pip install textattack
!pip install wordnet

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from scipy.spatial.distance import cosine
from nltk.corpus import wordnet
import os
from random import randint

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from transformers import BertTokenizer, TFBertModel
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')

In [ ]:
def get_synonyms(word):
    """    Get synonyms of a word    """
    synonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym)
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

In [ ]:
def replace_synonyms(text):
  #tokenize and split the text
  text = str(text).split() #CM - added str() here because some of the numbers in the text must have been coming through as ints rather than strings
  #rand pick the token to pick other than start and stop
  # -1 since
  token_num = randint(0,len(text)-1)
  #find all the synonyms for that token/words
  synonyms = get_synonyms(text[token_num])
  if len(synonyms) == 0:
    return ' '.join(text)
  else:
    text[token_num] = synonyms[0]
    return ' '.join(text)

In [ ]:
def bert_syn_substitution(translation_string,
                          MAX_SEQUENCE_LENGTH = 100):

  #tokenize the input string using the BERT tokenizer
  translation_string = translation_string.split()
  bert_train_tokenized = bert_tokenizer(translation_string,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')

  #turn the entire string into the bert embedding vector representation
  bert_output = bert_model(bert_train_tokenized)

  #pick a random token to use, add one to token num due to start token
  token_num = randint(0,len(translation_string))

  #this is our target embedding
  target_embedding = bert_output[0][0][token_num]

  #get the synonyms for the randomly chosen word, subtract one here to avoid list index error
  #token_num -=1
  candidates = get_synonyms(translation_string[token_num])

  #tokenize the potential synonym candiates
  candidates_tokenized = bert_tokenizer(candidates,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')

  #get the embeddings for the synonym candidates
  candidates_bert_output = bert_model(candidates_tokenized)

  #just need the embedding of the inputs the rest is padding
  candidates_bert_output = candidates_bert_output[0][0][:len(translation_string)]

  #create an empty list for the cosine similarities
  test= []
  #loop through all the embeddings appending the cosine similarity with the word we want to replace or target word
  for i in candidates_bert_output:
    cosine_similarity = 1 - cosine(i, target_embedding)
    test.append(cosine_similarity)


  #pick the word which had the highest cosines similarity
  word_to_replace_with = candidates[test.index(max(test))]

  #replace the word
  translation_string[token_num] = word_to_replace_with

  #return as combined string
  return ' '.join(translation_string)


In [ ]:
#for i in df_train['mt']:
#  df_train['mt'] = bert_syn_substitution(df_train['mt'].to_string())

In [ ]:
#for i in df_train['mt']:
#  print(df_train['mt'].to_string())

In [ ]:
#for i in df_train['mt']:
#  df_train['mt'] = bert_syn_substitution(i)

In [ ]:
df_train.shape

(73613, 10)

In [ ]:
##given the size of the train dataset lets set it up in batches
## ofc its a prime number so well the last row in order to chunk it
df_train = df_train.iloc[:-1 , :]

In [ ]:
n = int(df_train.shape[0]/4)

for i in range(0,df_train.shape[0],n):
    df_train['mt'][i:i+n] = bert_syn_substitution(df_train['mt'][i:i+n].to_string())

In [ ]:
# prompt: write a way to loog through items in a dataframe column which is very large to avoid memory issues

##given the size of the train dataset lets set it up in batches
## ofc its a prime number so well the last row in order to chunk it
df_train = df_train.iloc[:-1 , :]
n = int(df_train.shape[0]/22)

for i in range(0,df_train.shape[0],n):
    df_train['mt'][i:i+n] = bert_syn_substitution(df_train['mt'][i:i+n].to_string())

NameError: ignored